# Chapter 9: Triple exponential smoothing

The main disadvantage of the simple and double exponential smoothing models is that they do not recognise seasonal patterns and therefore cannot extrapolate any seasonal behavior in the future. To learn a seasonal pattern, we add a third layer of exponential smoothing, where the model will learn multiplicative seasonal factors that will be applied to each period inside a full seasonal cycle. As for the trend ($b$) and the level ($a$), the seasonal factors will be learned via an exponential weighting method with a new specific learning rate: $\gamma$.

Multiplicative seasonal factors mean, for xample, that the model will know the the demand is increased by 20% in January (compared to yearly average) but reduced by 30% in February.

## Model

The main idea is that the forecast is now composed of the level plus the damped trend, multiplied by the seasonal factor:
$$f_{t+1} = (a_t + \phi b_t) s_{t+1-p}.$$

The different demand components are calculated as such:

1. $$a_t = \alpha \frac{d_t}{s_{t-p}} + (1-\alpha)(a_{t-1} + \phi b_{t-1}),$$
2. $$b_t = \beta (a_t - a_{t-1}) + (1-\beta) \phi b_{t-1},$$
3. $$s_t = \gamma \frac{d_t}{a_t} + (1-\gamma)s_{t-p}.$$

Notice that both the level and the trend have been deseasonalised.

### Seasonal Factors Scaling

The seasonal factors determine how the forecast is allocated within an entire seasonal cycle. They should not impact the total demand of a full seasonal cycle, as on average, we will allocate 100% to each period. Mathematically, the sum of the seasonal factors over a full seasonal cycle should be $p$:
$$\sum_{cycle}s = p.$$

## Component initialization

The simple initialization technique for the double smoothing model was:
$$a_0 = d_0,$$
$$b_0 = d_1 - d_0.$$

Since the demand is now seasonal, these values need to be deseasonalised:
$$a_0 = \frac{d_0}{s_0},$$
$$b_0 = \frac{d_1}{s_1} - \frac{d_0}{s_0}.$$

There are discussions on how to initialize the seasonal factors for exponential smoothing algorithms. We use a simple method here: we initialize the seasonal factors based on the historical demand. We illustrate with the table below, which shows quarterly seasonal demand for over 5 years. The last column shows the average demand per quarter.

In [3]:
import pandas as pd


df = pd.DataFrame([{
    "Y1": 14, "Y2": 18, "Y3": 16, "Y4": 18, "Y5": 17, "Mean": 16.6
}, {
    "Y1": 10, "Y2": 8, "Y3": 9, "Y4": 11, "Y5": 9, "Mean": 9.4
},{
    "Y1": 6, "Y2": 4, "Y3": 5, "Y4": 4, "Y5": 5, "Mean": 4.8,
}, {
    "Y1": 2, "Y2": 1, "Y3": 3, "Y4": 2, "Y5": 1, "Mean": 1.8
}], index = ["Q1", "Q2", "Q3", "Q4"])

df

,Y1,Y2,Y3,Y4,Y5,Mean
Q1,14,18,16,18,17,16.6
Q2,10,8,9,11,9,9.4
Q3,6,4,5,4,5,4.8
Q4,2,1,3,2,1,1.8


The historical seasonal averages cannot be used idrectly as seasonal factors, as they are not scaled (the sum of seasonal factors should equal the period length). The seasonal means are scaled, by their own mean:

In [5]:
factor = df["Mean"].mean()
df["seasonal_factors"] = df["Mean"] / factor
df

,Y1,Y2,Y3,Y4,Y5,Mean,seasonal_factors
Q1,14,18,16,18,17,16.6,2.036810
Q2,10,8,9,11,9,9.4,1.153374
Q3,6,4,5,4,5,4.8,0.588957
Q4,2,1,3,2,1,1.8,0.220859


This can be interpreted as saying: "we sell 104% extra units in Q1, and we sell 15% extra in Q2, but -41% in Q3, and -78% in Q4".

With this triple exponential smoothing model, we can now deal with seasonal products, but the multiplicative part of the model might result in mathematical errors if the demand level or the seasonality levels are too close to 0. To solve this, next we work on a model with an additive seasonality.

In [11]:
import numpy as np

def initialize_seasonal_factors(time_series: list, s: np.array, period_length: int) -> np.array:
    """
    Uses the simple historical method to estimate the seasonal factors.
    The parameters s contains the newly initialized list which will contain seasonal factors, but is
    initially filled with np.nan values. This function will fill the first period_length values of s.
    """
    for i in range(period_length):
        s[i] = np.mean(time_series[i::period_length])
    s /= np.mean(s[:period_length])
    return s

def triple_exponential_smoothing(time_series: list, period_length: int = 12, horizon: int = 1, alpha: float = 0.4, beta: float = 0.4, phi: float = 0.9, gamma: float = 0.3):
    """
    Implements the triple exponential smoothing method to forecast horizon steps ahead.
    @input:
        time_series: list of time series values
        period_length: length of the seasonal period
        horizon: number of steps ahead to forecast
        alpha: smoothing parameter for the level
        beta: smoothing parameter for the trend
        phi: smoothing parameter for the seasonality
        gamma: smoothing parameter for the seasonal trend
    @output:
        forecasts: list of forecasts
    """

    number_historical_data = len(time_series)

    demand = np.append(time_series, np.full(horizon, np.nan))
    f = np.full(number_historical_data + horizon, np.nan)
    a = np.full(number_historical_data + horizon, np.nan)
    b = np.full(number_historical_data + horizon, np.nan)
    s = np.full(number_historical_data + horizon, np.nan)

    s = initialize_seasonal_factors(time_series, s, period_length)

    a[0] = demand[0] / s[0]
    b[0] = demand[1] / s[1] - demand[0] / s[0]

    for t in range(1, period_length):
        f[t] = (a[t-1] + phi*b[t-1]) * s[t]
        a[t] = alpha * demand[t] / s[t] + (1-alpha) * (a[t-1] + phi * b[t-1])
        b[t] = beta * (a[t] - a[t-1]) + (1-beta)*phi*b[t-1]

    for t in range(period_length, number_historical_data):
        f[t] = (a[t-1] + phi*b[t-1]) * s[t-period_length]
        a[t] = alpha * demand[t] / s[t-period_length] + (1-alpha) * (a[t-1] + phi * b[t-1])
        b[t] = beta * (a[t] - a[t-1]) + (1-beta)*phi*b[t-1]
        s[t] = gamma * demand[t] / a[t] + (1-gamma) * s[t-period_length]

    for t in range(number_historical_data, number_historical_data + horizon):
        f[t] = (a[t-1] + phi*b[t-1]) * s[t-period_length]
        a[t] = f[t] / s[t-period_length]
        b[t] = phi * b[t-1]
        s[t] = s[t-period_length]

    return pd.DataFrame({
        "demand": demand, "forecast": f, "level": a, "trend": b, "season": s, "error": demand - f
    })

data = [14,10,6,2,18,8,4,1,16,9,5,3,18,11,4,2,17,9,5,1]
df = triple_exponential_smoothing(data, period_length=12, horizon=4, alpha=0.3, beta=0.2, phi=0.9, gamma=0.2)
df

,demand,forecast,level,trend,season,error
0,14.0,NaN,7.145833,0.631944,1.959184,NaN
1,10.0,9.918750,7.733542,0.572542,1.285714,0.081250
2,6.0,5.050304,8.714180,0.608358,0.612245,0.949696
3,2.0,2.268172,8.933192,0.481820,0.244898,-0.268172
4,18.0,20.071778,9.076781,0.375628,2.142857,-2.071778
5,8.0,9.799125,8.896275,0.234351,1.040816,-1.799125
6,4.0,5.018248,8.552811,0.100040,0.551020,-1.018248
7,1.0,1.058308,8.499993,0.061465,0.122449,-0.058308
8,16.0,16.761427,8.438718,0.032000,1.959184,-0.761427
9,9.0,9.331551,8.377263,0.010749,1.102041,-0.331551


In [12]:
import support

support.kpi(df)

Bias: -0.19, -2.39%
MAPE: 9.30%
MAE: 0.64, 8.12%
RMSE: 0.92, 11.74%
